In [ ]:
#(Jupyter Notebook)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle

# Load the dataset
try:
    df = pd.read_csv('../Dataset/garments_worker_productivity.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: Dataset file 'garments_worker_productivity.csv' not found. Please ensure it's in the Dataset/ folder.")
    # Exit if file not found to prevent errors
    exit()

# Visualizing and Analyzing Data
print("Data head:")
print(df.head())
print("\nData info:")
print(df.info())

# Descriptive analysis
print("\nDescriptive Statistics:")
print(df.describe())

# Correlation analysis
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Garment Worker Productivity Data')
plt.show()

# Data Pre-processing
df['performance_label'] = (df['actual_productivity'] > df['targeted_productivity']).astype(int)
df = df.drop(columns=['date', 'actual_productivity', 'idle_time', 'idle_men', 'wip'])
df.dropna(inplace=True)
X = df.drop('performance_label', axis=1)
y = df['performance_label']
categorical_features = ['quarter', 'department', 'day']
numerical_features = X.drop(columns=categorical_features).columns

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Model Building
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)
print("\nModel Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save the model
joblib.dump(model_pipeline, 'gwp.pkl')
print("\nModel 'gwp.pkl' saved successfully in Training files.")

#save a pickle version
with open('mcle.pkl', 'wb') as file:
    pickle.dump(model_pipeline, file)
print("Model 'mcle.pkl' saved successfully in Training files.")